In [32]:
# importing libraries

import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder as LE
from sklearn import model_selection as msl
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as RFR

In [33]:
# importing data

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [34]:
# cleaning data

# creating data for test and tranfering data to be predicted
X = train_data.copy()
del X['SalePrice']
Y = train_data.SalePrice

X_test = test_data.copy()
del X_test['Id']
del X['Id']


# handling string values
for col in X.columns:
    if X[col].dtypes == 'object':
        d = {}
        itr = 0
        for j in range(len(X[col])):
            if X[col].iloc[j] in d:
                X[col].iloc[j] = d[X[col].iloc[j]]
            else:
                itr += 10
                d[X[col].iloc[j]] = itr
                X[col].iloc[j] = d[X[col].iloc[j]]
        for j in range(len(X_test[col])):
            if X_test[col].iloc[j] in d:
                X_test[col].iloc[j] = d[X_test[col].iloc[j]]
            else:
                itr += 10
                d[X_test[col].iloc[j]] = itr
                X_test[col].iloc[j] = d[X_test[col].iloc[j]]
                

# filling missing values
for col in X.columns:
    if X[col].isnull().sum():
        a = X[col].mean()
        X[col].fillna(a,inplace=True)
    if X_test[col].isnull().sum():
        a = X_test[col].mean()
        X_test[col].fillna(a,inplace = True)
        


X_test.head()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,50,80.0,11622,10,10,10,10,10,10,...,120,0,10,20,10,0,6,2010,10,10
1,20,10,81.0,14267,10,10,20,10,10,30,...,0,0,10,10,30,12500,6,2010,10,10
2,60,10,74.0,13830,10,10,20,10,10,10,...,0,0,10,20,10,0,3,2010,10,10
3,60,10,78.0,9978,10,10,20,10,10,10,...,0,0,10,10,10,0,6,2010,10,10
4,120,10,43.0,5005,10,10,20,40,10,10,...,144,0,10,10,10,0,1,2010,10,10


In [37]:
# feature scaling 
# but without feature scaling it gives better esults
for col in X.columns:
    a = X[col].mean()
    maxi = X[col].max()
    mini = X[col].min()
    for j in range(len(X[col])):
        X[col].iloc[j] = (X[col].iloc[j] - a)/(maxi - mini )
        
    a = X_test[col].mean()
    maxi = X_test[col].max()
    mini = X_test[col].min()
    for j in range(len(X_test[col])):
        X_test[col].iloc[j] = (X_test[col].iloc[j] - a)/(maxi - mini )
        
X.head()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.018251,-0.0851027,-0.017294,-0.009661,-0.00410959,-0.0452055,-0.136073,-0.0650685,-0.000684932,-0.149144,...,-0.031377,-0.003738,-0.00342466,-0.0832192,-0.0107877,-0.002806,-0.392902,0.046062,-0.0293664,-0.07
1,-0.217043,-0.0851027,0.034075,-0.004285,-0.00410959,-0.0452055,-0.136073,-0.0650685,-0.000684932,0.100856,...,-0.031377,-0.003738,-0.00342466,-0.0832192,-0.0107877,-0.002806,-0.120174,-0.203938,-0.0293664,-0.07
2,0.018251,-0.0851027,-0.007020,0.003427,-0.00410959,-0.0452055,0.19726,-0.0650685,-0.000684932,-0.149144,...,-0.031377,-0.003738,-0.00342466,-0.0832192,-0.0107877,-0.002806,0.243462,0.046062,-0.0293664,-0.07
3,0.077075,-0.0851027,-0.034418,-0.004519,-0.00410959,-0.0452055,0.19726,-0.0650685,-0.000684932,0.350856,...,-0.031377,-0.003738,-0.00342466,-0.0832192,-0.0107877,-0.002806,-0.392902,-0.453938,-0.0293664,0.13
4,0.018251,-0.0851027,0.047774,0.017496,-0.00410959,-0.0452055,0.19726,-0.0650685,-0.000684932,0.100856,...,-0.031377,-0.003738,-0.00342466,-0.0832192,-0.0107877,-0.002806,0.516189,0.046062,-0.0293664,-0.07


In [38]:
# for testting purpose only 
# check which algo has more score
algo = LinearRegression()
algo2 = RFR(max_depth = 200)
X_t,X_v,Y_t,Y_v = msl.train_test_split(X,Y,random_state = 5)

algo.fit(X_t,Y_t)
algo2.fit(X_t,Y_t)

print(algo.score(X_v,Y_v))
print(algo2.score(X_v,Y_v))

0.8244284534422873
0.8816583082409362


In [36]:
# whichever algo has more score predict according to that
algo2.fit(X,Y)
preds = algo2.predict(X_test)

In [28]:
ans = {
    'ID':test_data['Id'],
    'SalePrice':preds
}
ans = pd.DataFrame(ans)
ans.to_csv('pred.csv',index = False)